In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import load_params

def plot_square(x_indices, y_indices, ax):
    ax.plot([x_indices[0], x_indices[-1]], [y_indices[-1],y_indices[-1]], color='red', linewidth=1)
    ax.plot([x_indices[0], x_indices[-1]], [y_indices[0], y_indices[0]], color='red', linewidth=1)
    ax.plot([x_indices[0], x_indices[0]], [y_indices[0],  y_indices[-1]], color='red', linewidth=1)
    ax.plot([x_indices[-1],x_indices[-1]], [y_indices[0], y_indices[-1]], color='red', linewidth=1)

# Experimental distance-frequency matrix

# 1. Frequency slices

In [ ]:
exp_name = '2020_12_9_rotating'; 
fname = f'results/{exp_name}_real.pkl'

try:
    df_freq = pd.read_pickle(fname)
    print('read', fname)
except:
    print('Error: run wall_analysis.py to parse experiments.')
df_freq.iloc[:, :8]

In [ ]:
from simulation import get_setup

distance = 10

fig = plt.figure()
fig.set_size_inches(5, 5)
for yaw_deg, marker in zip([0, 30], ['o', 'x']):
    source, mic_positions = get_setup(distance_cm=10, yaw_deg=yaw_deg)
    source_image = [source[0], -source[1]]
    for i, mic in enumerate(mic_positions):
        plt.scatter(*mic, label=f'mic{i}, {yaw_deg}deg', marker=marker, color=f'C{i}')
plt.plot([4.8, 5.4], [0, 0], color='k', label='wall')
plt.title(f'setup for distance={distance}cm')
plt.legend(loc='upper right')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')

fig.savefig(f'plots/setup_{distance}.pdf', bbox_inches='tight')

### spectrograms

In [ ]:
from frequency_analysis import get_spectrogram_raw

mic_type = 'measurement'
cut_x = range(0, 20)
cut_y = range(1000, 3000)

filters = ['mic_type', 'snr', 'source', 'degree', 'distance']
for chosen_tuple, df_source in df_freq.groupby(filters):
    
    fig, axs = plt.subplots(1, 2, squeeze=False)
    fig.set_size_inches(15, 5)
    fig.suptitle(dict(zip(filters, chosen_tuple)))
    
    for j, motors in enumerate([0, "all43000"]):
        df_this = df_source[df_source.motors==motors]
        if len(df_this) != 1:
            print(df_this)
            continue
        row = df_this.iloc[0]
        
        spec_all, freqs = get_spectrogram_raw(row.frequencies_matrix, row.stft)
        spec = np.mean(spec_all, axis=1) # n_times x n_mics x n_freqs
        
        axs[0, j].pcolorfast(row.seconds, freqs, np.log10(spec[:-1, :-1]))
        axs[0, j].set_title(f'motors {motors}')

    #spec_cut = spec[:, cut_x]
    #spec_cut = spec_cut[cut_y, :]
    #axs[0, 2].pcolorfast(row.seconds[cut_x], row.frequencies[cut_y], np.log10(spec_cut))
    #plot_square(cut_x, cut_y, ax=axs[0, 1])
    #fig.savefig(f'/home/duembgen/Desktop/spec-{chosen_source}.png', bbox_inches='tight')

### frequency slices

### 1. fix psd parameters for frequency sweeps

In [ ]:
from wall_analysis import filter_by_dicts
from frequency_analysis import apply_linear_mask, apply_box_mask
from frequency_analysis import psd_df_from_spec, get_index_matrix, extract_psd

chosen_dicts = [{"appendix":"", "degree": 0, "distance":10, "motors":0}]
#chosen_dicts = [{"appendix":"", "snr": 1, "degree": 0, "distance":10, "motors":0}]
df_mic_types = filter_by_dicts(df_freq, chosen_dicts)

kwargs_all = {
    'audio_deck': {
        'slope': 4000/170,
        'offset': 400,
        'delta': 20,
        'min_freq': 2000,
        'max_freq': 5000,
        'min_time': 70,
        'max_time': 190, 
        'mics': range(4)
    },
    'measurement': {
        'slope': 4000/510,
        'offset': 600,
        'delta': 20,
        'min_freq': 2000,
        'max_freq': 5000,
        'min_time': 190,
        'max_time': 520,
        'mics': [1]
    }
}

fig_all, ax_all = plt.subplots()

for j, (i_row, row) in enumerate(df_mic_types.iterrows()):
    # get raw spectrogram
    spec_all, freqs = get_spectrogram_raw(row.frequencies_matrix, row.stft)
    spec = np.mean(spec_all, axis=1)
    
    kwargs = kwargs_all[row.mic_type]

    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.pcolorfast(range(spec.shape[1]), freqs, np.log10(spec[:-1, :-1]))

    # apply mask to spec
    spec_masked_all, freqs = apply_linear_mask(spec_all, freqs, 
                                               kwargs['slope'], 
                                               kwargs['offset'], 
                                               kwargs['delta'], ax=ax)
    spec_masked_all, freqs = apply_box_mask(spec_masked_all, freqs, 
                                            min_freq=kwargs['min_freq'], 
                                            max_freq=kwargs['max_freq'], 
                                            min_time=kwargs['min_time'], 
                                            max_time=kwargs['max_time'], 
                                            ax=ax)

    # get frequency slice
    index_matrix = get_index_matrix(spec_masked_all)
    ax.plot(freqs[index_matrix[0, :]], color='white', ls=':')
    
    psd_df = psd_df_from_spec(spec_masked_all, freqs, index_matrix)
    psd, freqs_psd, std_psd = extract_psd(psd_df)

    plt.figure()
    for i in range(psd.shape[0]):
        plt.errorbar(freqs_psd, psd[i, :], std_psd[i, :], label=f'mic{i}')
    [plt.axvline(f, color='black', ls=':') for f in freqs_psd]
    
    [ax_all.plot([f, f], [j, j+1], color=f'C{j}', ls=':') for f in freqs_psd]
    ax_all.plot([], [], color=f'C{j}', ls=':', label=f'{row.mic_type}, snr={row.snr}')
    
    plt.yscale('log')
    plt.legend()
    
from bin_selection import generate_sweep
from crazyflie_description_py.parameters import N_BUFFER, FS
bins, t_sec = generate_sweep('sweep')
freqs = np.fft.rfftfreq(N_BUFFER, 1/FS)
[ax_all.plot([f, f], [j+1, j+2], color=f'C{j+1}', ls='-') for f in freqs[bins]]
ax_all.plot([], [], color=f'C{j+1}', ls=':', label='sweep')
ax_all.legend(loc='lower left')
pass


### 2. apply above to full dataset

In [ ]:
from bin_selection import select_frequencies
from constants import SPEED_OF_SOUND
from crazyflie_description_py.parameters import N_BUFFER
    
params_freq = load_params(exp_name)
    
filters = ['snr', 'motors', 'mic_type', 'distance']
frequency_slice_df = pd.DataFrame(columns=filters+['psd', 'freqs', 'psd_std'])

for chosen_tuple, df_this in df_freq.groupby(filters):
    filter_dict = dict(zip(filters, chosen_tuple))
    if filter_dict['motors'] != 0:
        continue
        
    mic_type = filter_dict['mic_type']
    kwargs = kwargs_all[mic_type]
    mic_indices = kwargs_all[mic_type]['mics']

    fig, axs = plt.subplots(1, len(mic_indices), squeeze=False)
    fig.set_size_inches(10, 5)
    fig.suptitle(filter_dict)

    if len(df_this) != 1: 
        print(f"{len(df_this)} findings for {filter_dict}")
        continue

    row = df_this.iloc[0]
    
    spec_all, freqs = get_spectrogram_raw(row.frequencies_matrix, row.stft)
    spec_masked_all, freqs = apply_linear_mask(spec_all, freqs, 
                                               kwargs['slope'], 
                                               kwargs['offset'], 
                                               kwargs['delta'])#, ax=ax)
    spec_masked_all, freqs = apply_box_mask(spec_masked_all, freqs, 
                                            min_freq=kwargs['min_freq'], 
                                            max_freq=kwargs['max_freq'],
                                            min_time=kwargs['min_time'], 
                                            max_time=kwargs['max_time'],
                                           )#, ax=ax)
    index_matrix = get_index_matrix(spec_masked_all)
    psd_df = psd_df_from_spec(spec_masked_all, freqs, index_matrix)
    psd, freqs_psd, std_psd = extract_psd(psd_df)

    for i, mic_idx in enumerate(mic_indices):
        axs[0, i].errorbar(freqs_psd, psd[i], std_psd[i], 
                          label=f'mic{mic_idx}', color=f'C{mic_idx}')
        axs[0, i].set_yscale('log')
        axs[0, i].set_title(f'm{mic_idx}')
        
    df_data = filter_dict
    df_data.update({'psd':psd, 'freqs':freqs_psd, 'psd_std':std_psd})
    frequency_slice_df.loc[len(frequency_slice_df), :] = df_data

### 3. calibration

In [ ]:
from scipy.interpolate import interp1d

CHOSEN_MICS = [0, 1, 2, 3] #[0, 1, 2, 3] #[3]

calib_df = pd.read_pickle('results/calibration_results.pkl')
chosen_dict = {
   'source': 'sweep',
   #'method': 'median-reject',
   'method': 'median',
   'type': 'psd_df',
}
calib_df = filter_by_dicts(calib_df, [chosen_dict])
assert len(calib_df) == 1
row = calib_df.iloc[0]

fig, axs = plt.subplots(1, len(CHOSEN_MICS), squeeze=False, sharey=True, sharex=True)
fig.set_size_inches(10, 5)
#fig.suptitle(f'{distance}cm, {mic_type} mic, motors:{motors}')
for i, mic_idx in enumerate(CHOSEN_MICS):
    axs[0, i].semilogy(row.frequencies, row.psd[mic_idx], color=f'C{mic_idx}')
    axs[0, i].set_xlabel('frequency [Hz]')
    axs[0, i].set_title(f'mic{mic_idx}')
axs[0, 0].set_ylabel('PSD')
axs[0, i].set_ylim(1e-2, 3e2)

calib_psd = row.psd
calib_f = row.frequencies

uniform_f = np.linspace(1000, 6000, 50)
calib = interp1d(x=calib_f, y=calib_psd, kind='linear', fill_value='extrapolate')
uniform_psd = calib(uniform_f)
for i, mic_idx in enumerate(CHOSEN_MICS):
    axs[0, i].scatter(uniform_f, uniform_psd[mic_idx], color=f'C{mic_idx}')
    axs[0, i].grid('both')
    
fig.savefig('plots/calibration.pdf', bbox_inches='tight')

In [ ]:
from simulation import get_freq_slice_pyroom, get_freq_slice_theory
    
for chosen_tuple, df_this in frequency_slice_df.groupby(filters):
    filter_dict = dict(zip(filters, chosen_tuple))
    kwargs = kwargs_all[filter_dict['mic_type']]
    
    if filter_dict['snr'] != 1:
        continue
    
    if len(df_this) != 1: 
        print(f"{len(df_this)} findings for {filter_dict}")
        continue
    row = df_this.iloc[0]
    
    psd = row.psd
    psd_std = row.psd_std
    frequencies = row.freqs
    psd_calib = calib(frequencies)
    
    fig, axs = plt.subplots(3, psd.shape[0], squeeze=False, sharey=True, sharex=True)
    axs[0, 0].set_yscale('log')
    axs[0, 0].set_ylim(1e-2, 3e2)
    fig.set_size_inches(10, 10)
    
    slices_theory = get_freq_slice_theory(frequencies, distance_cm=filter_dict['distance'])
    slices_pyroom = get_freq_slice_pyroom(frequencies, distance_cm=filter_dict['distance'])
    
    for j, mic_idx in enumerate(kwargs['mics']):
        axs[0, j].set_title(f'm{mic_idx}')
        axs[0, j].errorbar(frequencies, psd[j], psd_std[j], label='|Y(f)|', color=f'C{mic_idx}')
        axs[0, j].plot(frequencies, psd_calib[j], ls=':', label='|X(f)|', color=f'C{mic_idx}')
        #axs[0, j].plot(frequencies, psd_mean, color='k')
        
        psd_ratio = psd[j]/psd_calib[j]
        axs[1, j].plot(frequencies, psd_ratio, label=f'|H(f)|', color=f'C{mic_idx}')
        #axs[1, j].plot(frequencies, psd[j]/psd_mean, label=f'global calib', color='k')
        
        axs[0, j].legend(loc='lower right')
        axs[1, j].legend(loc='lower right')
        axs[2, j].plot(frequencies, np.abs(slices_theory[mic_idx]), label='theory', color="black")
        axs[2, j].plot(frequencies, np.abs(slices_pyroom[mic_idx]), label='pyroom', color="gray")
        axs[2, j].legend(loc='lower right')
        
        axs[0, j].grid('both')
        axs[1, j].grid('both')
        axs[2, j].grid('both')

# Distance slices

In [ ]:
#exp_name = '2020_12_7_moving'; 
exp_name = '2020_12_18_stepper'; 
fname = f'results/{exp_name}_real.pkl'

params_dist = load_params(exp_name)

try:
    df_dist = pd.read_pickle(fname)
    print('read', fname)
except:
    print('Error: run wall_analysis.py to parse experiments.')

In [ ]:
from wall_analysis import add_distance_estimates
from frequency_analysis import add_spectrogram

df_dist = df_dist.assign(d_estimate=None)
df_dist = df_dist.apply(add_distance_estimates, axis=1)

## sanity checks

In [ ]:
import itertools
from wall_analysis import filter_by_dicts
labels = ['motors', 'source']
vals_dict = {l:df_dist[l].unique() for l in labels}
vals_dict['source'] = sorted(vals_dict['source'])
vals_dict['motors'] = sorted([m for m in vals_dict['motors'] if type(m) is int]) + sorted([m for m in vals_dict['motors'] if not type(m) is int])
print(vals_dict)

fig, axs = plt.subplots(1, len(df_dist.mic_type.unique()))
fig.set_size_inches(10, 5)
axs = dict(zip(df_dist.mic_type.unique(), axs))
fig.suptitle('timestamps for different datasets')
for mic_type, df_mic in df_dist.groupby('mic_type'):
    for i, val_tuple in enumerate(itertools.product(*vals_dict.values())):
        filter_dict = dict(zip(labels, val_tuple))
        df_here = filter_by_dicts(df_mic, [filter_dict])
        
        label = f"{list(filter_dict.values())}"
        if len(df_here) != 1:
            axs[mic_type].plot([], [], color=f'C{i%10}', label=label + ': missing')
            continue
        row = df_here.iloc[0]
        axs[mic_type].plot(row.seconds, row.seconds+i*10, color=f'C{i%10}', label=label)
    axs[mic_type].set_title(mic_type)
    axs[mic_type].legend(bbox_to_anchor=[1.0, -0.1], loc='upper right')
    
fig, axs = plt.subplots(1, len(df_dist.mic_type.unique()))
fig.set_size_inches(10, 5)
axs = dict(zip(df_dist.mic_type.unique(), axs))
fig.suptitle('distance estimates for different datasets')
for mic_type, df_mic in df_dist.groupby('mic_type'):
    for i, val_tuple in enumerate(itertools.product(*vals_dict.values())):
        filter_dict = dict(zip(labels, val_tuple))
        df_here = filter_by_dicts(df_mic, [filter_dict])
        
        label = f"{list(filter_dict.values())}"
        if not len(df_here):
            axs[mic_type].plot([], [], color=f'C{i%10}', label=label + ': missing')
            continue
        elif len(df_here) > 2:
            raise ValueError(len(df_here))
        row = df_here.iloc[0]
        axs[mic_type].plot(row.seconds, row.d_estimate+i/100, color=f'C{i%10}', label=label)
    axs[mic_type].set_title(mic_type)
    axs[mic_type].legend(bbox_to_anchor=[1.0, -0.1], loc='upper right')

In [ ]:
from wall_analysis import filter_by_dicts
print(exp_name)
if exp_name == "2020_12_7_moving":
    motors = "all43000"
    chosen_dicts = [
        {"source": "mono3125", "motors": 0, "appendix":""},
        {"source": "mono3125", "motors": motors, "appendix":"_new"},
        {"source": "mono4156", "motors": 0, "appendix":""},
        {"source": "mono4156", "motors": motors, "appendix":"_new"},
        {"source": "mono8000", "motors": 0, "appendix":"_new"},
        {"source": "mono8000", "motors": motors, "appendix":""},
        {"source": "None", "motors": motors, "appendix":"_new"},
    ]
    df_chosen = filter_by_dicts(df_dist, chosen_dicts)
else: 
    chosen_dicts = [
        {}
    ]
    df_chosen = filter_by_dicts(df_dist, chosen_dicts)
print(df_chosen.source.unique())

## spectrograms

In [ ]:
from frequency_analysis import get_spectrogram_raw

filters = ['source', 'degree', 'mic_type', 'snr']
for chosen_tuple, df_source in df_chosen.groupby(filters):
    filter_dict = dict(zip(filters, chosen_tuple))
    fig, axs = plt.subplots(1, 3, squeeze=False)
    fig.set_size_inches(15, 5)
    title = ''
    for key, val in filter_dict.items():
        title += f'{key}: {val}, '
        break
    title = title[:-2]
    fig.suptitle(title)
    
    j_square = None
    for j, motors in enumerate(params_dist.MOTORS_LIST):
        df_this = df_source[df_source.motors==motors]
        if len(df_this) != 1:
            continue
        row = df_this.iloc[0]
        
        spec_all, freqs = get_spectrogram_raw(row.frequencies_matrix, row.stft)
        spec = np.mean(spec_all, axis=1)

        axs[0, j].pcolorfast(row.seconds, freqs, np.log10(spec[:-1, :-1]))
        axs[0, j].set_title(f'motors {motors}')
        axs[0, j].set_ylim(100, 5000)
        j_square = j
        
    #if j_square is not None:
        #plot_square(row.seconds, row.frequencies, cut_x, cut_y, axs[0, j_square]) 
        #spec_square  = spec[:, cut_x]
        #spec_square = spec_square[cut_y, :]
        #axs[0, 2].pcolormesh(row.seconds[cut_x], row.frequencies[cut_y], np.log10(spec_square))
    axs[0, 0].set_xlabel('time idx')
    axs[0, 0].set_ylabel('frequency [Hz]')
    axs[0, 1].set_xlabel('time idx')
    axs[0, 1].set_ylabel('frequency [Hz]')
    axs[0, 2].set_xlabel('time idx')
    axs[0, 2].set_ylabel('frequency [Hz]')
    
    #fname = f'plots/spec_{filter_dict["source"]}.pdf' 
    #fig.savefig(fname, bbox_inches='tight')
    #print('saved as', fname)

## frequency selection for propeller noise

In [ ]:
df_source = df_chosen.loc[df_chosen.source=="None"]
df_mic = df_source.loc[df_source.mic_type == "audio_deck"]
assert len(df_mic) == 1, df_mic
row = df_mic.iloc[0]

f_matrix = row.frequencies_matrix[:, 1:]
print(f_matrix.shape)
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
for i, col in enumerate(f_matrix.T):
    if i < 3:
        plt.scatter(row.seconds, col, color=f'C{i}', s=2)
    else:
        plt.scatter(row.seconds, col, color='C1', s=1, alpha=0.5)
plt.ylim(100, 2000)

levels = list(range(5))
num_levels = len(levels)
f_matrix
fig, axs = plt.subplots(num_levels, sharey=False, sharex=True, squeeze=False)
fig.set_size_inches(10, 10)
for j, level in enumerate(levels):
    axs[j, 0].plot(f_matrix[:, level], color=f"C{j}", marker='o')
    axs[j, 0].set_title(f'level {level}')
axs[j, 0].set_xlabel(f'time [idx]')

In [ ]:
all_freqs = np.sort(f_matrix[:, :10].flatten())
count, bins = np.histogram(all_freqs, bins=np.unique(all_freqs))
plt.plot(bins[:-1], count, marker='x')
plt.title('histogram strongest 10 frequency bins')
print(bins[np.argsort(count)[::-1]])

## distance-slices

In [ ]:
#mic_indices = range(4)
from constants import SPEED_OF_SOUND
from frequency_analysis import get_bin

filters = ['snr', 'motors', 'mic_type', 'distance']
frequency_slice_df = pd.DataFrame(columns=filters+['psd', 'freqs', 'psd_std'])

CHOSEN_MICS = [0, 1, 2, 3] #[3]
CUT_LAST = 3
CUT_FIRST = 3

#CHOSEN_PROP_FREQ = 781 #
CHOSEN_PROP_FREQ = 671 #
#CHOSEN_PROP_FREQ = 2187 #1
#CHOSEN_PROP_FREQ = 1453 

for (chosen_source, mic_type), df_this in df_chosen.groupby(['source', 'mic_type']):
    fig, axs = plt.subplots(1, 2, squeeze=False)
    fig.set_size_inches(15, 5)
    fig.suptitle(chosen_source)

    for j, motors in enumerate(params_dist.MOTORS_LIST):
        df_motors = df_this.loc[df_this.motors==motors]

        if len(df_motors) != 1: 
            print(f"{len(df_motors)} findings for {chosen_source, motors, mic_type}")
            continue
            
        row = df_motors.iloc[0]
        if (chosen_source == "None"):
            freq = CHOSEN_PROP_FREQ
        elif "mono" in chosen_source:
            freq = int(chosen_source.strip('mono'))
        else:
            print('warning: using 1000Hz', chosen_source)
            freq = 1000

        spec, freqs = get_spectrogram_raw(row.frequencies_matrix, row.stft)
        bin_ = get_bin(freqs, freq)

        #seconds = row.seconds[CUT_FIRST:-CUT_LAST]
        #distances = seconds * 50 / 165.0
        distances = row.d_estimate[CUT_FIRST:-CUT_LAST] * 100
        for mic_idx in range(spec.shape[1]):
            spec_bin = spec[bin_, mic_idx, CUT_FIRST:-CUT_LAST]
            axs[0, j].plot(distances, spec_bin, label=f'mic{mic_idx}', color=f'C{mic_idx}')
            axs[0, j].set_title(f'motors {motors}')

        axs[0, j].set_ylim(min(spec_bin), max(spec_bin))
        axs[0, j].legend(loc='upper right')
        axs[0, j].set_xlim(10,60)
        #axs[0, j].set_ylim(0,min(1, np.max(spec)))
        continue

        mean_d = np.median(distances[5:]-distances[4:-1])
        #print(mean_d)
        expected_period = SPEED_OF_SOUND / freq * 100 / 2 #
        for mic_idx in mic_indices:
            spec_fft = np.fft.rfft(spec)[1:]
            freqs = np.fft.rfftfreq(len(spec), mean_d)[1:]
            period = 1/freqs
            axs[1, j].semilogx(period, np.abs(spec_fft), color=f'C{mic_idx}')
            axs[1, j].axvline(expected_period, color=f'C{mic_idx}', ls=":") # cm
        axs[1, j].set_xlim(1e-1,50)

## study chosen datasets more carefully

In [ ]:
frequency = 1750 
#frequency = 3125 
#frequency = 2375
filter_dict = {
    'source': f'mono{frequency}',
    #'motors': 0,
    'motors': 'all45000',
    'mic_type': 'audio_deck',
    #'mic_type': 'measurement'
}
df_chosen = filter_by_dicts(df_dist, [filter_dict])
assert len(df_chosen) == 1, len(df_chosen)
row = df_chosen.iloc[0]
n_mics = row.stft.shape[1]

In [ ]:
from simulation import get_dist_slice_pyroom, get_dist_slice_theory

spec, freqs = get_spectrogram_raw(row.frequencies_matrix, row.stft)
n_mics = spec.shape[1]

bin_ = get_bin(freqs, frequency) 

distances_cm = np.linspace(10, 60)
slices_theory = get_dist_slice_theory(frequency, distances_cm=distances_cm)
slices_pyroom = get_dist_slice_pyroom(frequency, distances_cm=distances_cm)

fig1, axs1 = plt.subplots(1, n_mics, squeeze=False, sharey=True)
fig2, axs2 = plt.subplots(1, n_mics, squeeze=False, sharey=True)
fig1.set_size_inches(10, 5)
fig2.set_size_inches(10, 5)

for mic_idx in range(n_mics):
    mask = ~np.isnan(row.d_estimate)
    axs1[0, mic_idx].set_title(f'mic{mic_idx}')
    axs1[0, mic_idx].semilogy(row.d_estimate[mask], spec[bin_, mic_idx, mask], 
                      color=f'C{mic_idx}', label='|Y(d)|')
    axs1[0, mic_idx].grid(which='both')
    axs1[0, mic_idx].legend(loc='upper left')
    axs1[0, mic_idx].set_xlabel('distance [m]')
    
    axs2[0, mic_idx].semilogy(distances_cm / 100, slices_theory[:, mic_idx], color='black', label='theory')
    axs2[0, mic_idx].semilogy(distances_cm / 100, slices_pyroom[:, mic_idx], color='gray', label='pyroom')
    axs2[0, mic_idx].grid(which='both')
    axs2[0, mic_idx].set_xlabel('distance [m]')
    axs2[0, mic_idx].legend(loc='upper left')
    axs2[0, mic_idx].set_ylim(2, 20)
    
    # custom stuff
    if frequency == 1750:
        if row.mic_type == 'measurement':
            #axs1[0, mic_idx].set_ylim(1e-3, 1e-1)
            #axs1[0, mic_idx].set_xlim(0.2, 0.4)
            #axs2[0, mic_idx].set_xlim(0.2, 0.4)
            pass
        elif row.mic_type == 'audio_deck':
            #axs1[0, mic_idx].set_ylim(1e-3, 5)
            axs1[0, mic_idx].set_xlim(0.1, 0.25)
            axs2[0, mic_idx].set_xlim(0.1, 0.25)
            pass
    if frequency == 2375:
        if row.mic_type == 'measurement':
            #axs1[0, mic_idx].set_ylim(1e-3, 1e-1)
            axs1[0, mic_idx].set_xlim(0.2, 0.4)
            axs2[0, mic_idx].set_xlim(0.2, 0.4)
        elif row.mic_type == 'audio_deck':
            #axs1[0, mic_idx].set_ylim(1e-3, 5)
            axs1[0, mic_idx].set_xlim(0.3, 0.5)
            axs2[0, mic_idx].set_xlim(0.3, 0.5)
    elif frequency == 3125:
        if row.mic_type == 'measurement':
            #axs1[0, mic_idx].set_ylim(1e-3, 1e-1)
            axs1[0, mic_idx].set_xlim(0.1, 0.3)
            axs2[0, mic_idx].set_xlim(0.1, 0.3)
        elif row.mic_type == 'audio_deck':
            #axs1[0, mic_idx].set_ylim(1e-3, 5)
            axs1[0, mic_idx].set_xlim(0.3, 0.5)
            axs2[0, mic_idx].set_xlim(0.3, 0.5)
    
fig1.suptitle(f'{frequency} Hz, {row.mic_type} mic, motors:{row.motors}')
fname = f'plots/d_slice_{frequency}_{row.mic_type}_{row.motors}.pdf'
fig1.savefig(fname, bbox_inches='tight')

fname = f'plots/d_slice_{frequency}_{row.mic_type}_{row.motors}_theory.pdf'
fig2.savefig(fname, bbox_inches='tight')
print('saved as', fname)

In [ ]:
# 1. study the spectrum, frequency selection
min_freq = 100
max_freq = 5000

spec_all, all_frequencies = get_spectrogram_raw(row.frequencies_matrix, row.stft)
spec = np.nanmean(spec_all, axis=1)

mask = (all_frequencies > min_freq) & (all_frequencies < max_freq)
freqs = all_frequencies[mask]

fig, ax = plt.subplots()
plot_spec = np.full(spec.shape, np.nan)
plot_spec[spec>0] = np.log10(spec[spec>0])
ax.pcolorfast(range(len(row.seconds)), freqs, plot_spec[mask][:-1, :-1])
ax.axhline(frequency, color='black', ls=':')

fig, axs = plt.subplots(1, n_mics, squeeze=False)
fig.set_size_inches(10, 5)
for mic_idx in range(n_mics):
    spec_avg = np.nanmean(spec_all[mask, mic_idx, :], axis=-1)
    axs[0, mic_idx].plot(freqs, spec_avg, color=f'C{mic_idx}')
    axs[0, mic_idx].axvline(frequency, color='black', ls=':')
    freq_max = freqs[np.argmax(spec_avg)]
    print(f'max bin vs. played bin: {freq_max}, {frequency}, difference: {freq_max - frequency}')

# Old experiments

In [ ]:
from evaluate_data import read_df, integrate_yaw
from dynamic_analysis import add_pose_to_df

exp_name = '2020_11_26_wall'; 

fname = f'results/{exp_name}_real.pkl'

try:
    df_total = pd.read_pickle(fname)
    frequencies = df_total.iloc[0].frequencies
    print('read', fname)
except:
    print('could not read', fname)
    print('run wall_analysis.py')

In [ ]:
from wall_detector import WallDetector
import progressbar

df_total.tail()

kwargs = {
    'slope': (4000 - 1000) / 200,
    'offset': 200,
    'delta': 50
}

wall_detector = WallDetector(kwargs)

psd_df = pd.DataFrame

with progressbar.ProgressBar(max_value = len(df_total)) as p:
    for i, row in df_total.iterrows():
        spec, freqs = wall_detector.fill_from_raw(row.frequencies_matrix, row.stft, distance=row.distance)
        if i == 0:
            fig, ax = plt.subplots()
            spec_avg = np.mean(spec, axis=1) 
            ax.pcolorfast(np.arange(spec.shape[2]), freqs, np.log10(spec_avg))
        p.update(i)

In [ ]:
f_slices, freqs = wall_detector.get_frequency_slice(distance=0)
plt.figure()
plt.semilogy(freqs, f_slices[0])

In [ ]:
d_slices, distances = wall_detector.get_distance_slice(frequency=1125)
plt.figure()
plt.plot(distances, d_slices[3])

In [ ]:
distances, freqs, df_matrix_mics = wall_detector.get_df_matrix()

In [ ]:
from audio_stack.beam_former import normalize_rows

df_matrix_norm = np.zeros_like(df_matrix_mics)

for row in range(df_matrix_mics.shape[1]):
    for mic in range(df_matrix_mics.shape[0]):
        sub = df_matrix_mics[mic, row, :] 
        # 0 to 1
        range_ = np.max(sub) - np.min(sub)
        sub_norm = (sub - np.min(sub)) / range_ 
        df_matrix_norm[mic, row, :] = sub_norm

        # sum to 1
        #sub_norm = sub / np.sum(sub)
        #df_matrix_norm[mic, row, :] = sub_norm
    
#mat = np.transpose(df_matrix_mics, (1, 2, 0))
#mat_norm = normalize_rows(mat, method='sum_to_one')
#df_matrix_norm = np.transpose(mat_norm, (2, 0, 1))

for i in range(df_matrix_mics.shape[0]):
    fig, ax = plt.subplots()
    ax.pcolorfast(distances, freqs, np.log10(df_matrix_mics[i, :-1, :-1]))
    ax.set_title(f'mic{i}')
    
    fig, ax = plt.subplots()
    ax.pcolorfast(distances, freqs, np.log10(df_matrix_norm[i, :-1, :-1]))
    ax.set_title(f'mic{i} normalized')

## distance slices

In [ ]:
from constants import SPEED_OF_SOUND

for slice_f in range(df_matrix_norm.shape[1]):
    fig, axs1 = plt.subplots(1, wall_detector.n_mics, sharey=False)
    fig.set_size_inches(15, 2)
    fig, axs2 = plt.subplots(1, wall_detector.n_mics, sharey=True)
    fig.set_size_inches(15, 2)
    axs = np.r_[[axs1, axs2]]
    
    f = frequencies[slice_f]
    expected_period = SPEED_OF_SOUND / f * 1e2 / 2
    
    for mic_idx in range(wall_detector.n_mics):
        distance_response = df_matrix_norm[mic_idx, slice_f, :]
        axs[0, mic_idx].semilogy(distances, distance_response, label=f'{f:.0f}Hz')
        axs[0, mic_idx].set_xlabel('distance [cm]')
        axs[0, mic_idx].set_title(f'mic{mic_idx}')
        axs[0, mic_idx].legend(loc='lower left')

        distance_fft = np.fft.rfft(distance_response)[1:]
        distance_freq = 1 / np.fft.rfftfreq(len(distances), 1)[1:] # cm 
        axs[1, mic_idx].loglog(distance_freq, np.abs(distance_fft))
        axs[1, mic_idx].axvline(expected_period, ls=":")
        axs[1, mic_idx].set_xlabel('period [cm]')
    
    axs[0, 0].set_ylabel('PSD')
    axs[1, 0].set_ylabel('mag. of FFT of PSD')

## frequency slices

In [ ]:
from constants import SPEED_OF_SOUND

for slice_d in range(1, df_matrix_norm.shape[2]):
    fig, axs1 = plt.subplots(1, wall_detector.n_mics, sharey=False)
    fig.set_size_inches(15, 2)
    fig, axs2 = plt.subplots(1, wall_detector.n_mics, sharey=True)
    fig.set_size_inches(15, 2)
    axs = np.r_[[axs1, axs2]]
    
    d = distances[slice_d]
    
    if d > 0:
        expected_period = SPEED_OF_SOUND / (d * 1e-2) / 2
    else: 
        expected_period = 0
    
    for mic_idx in range(wall_detector.n_mics):
        frequency_response = df_matrix_norm[mic_idx, :, slice_d]
        mask = ~np.isnan(frequency_response)
        frequency_response = frequency_response[mask]
        
        axs[0, mic_idx].semilogy(frequencies[mask], frequency_response, label=f'{d:.0f}cm')
        axs[0, mic_idx].set_xlabel('frequency [Hz]')
        axs[0, mic_idx].set_title(f'mic{mic_idx}')
        axs[0, mic_idx].legend(loc='lower left')

        frequency_fft = np.fft.rfft(frequency_response)[1:]
        mean_df = np.mean(frequencies[mask][1:]-frequencies[mask][:-1])
        frequency_freq = 1 / np.fft.rfftfreq(len(frequencies[mask]), mean_df)[1:] # cm 
        axs[1, mic_idx].loglog(frequency_freq, np.abs(frequency_fft))
        axs[1, mic_idx].axvline(expected_period, ls=":")
        axs[1, mic_idx].set_xlabel('period [Hz]')
    
    axs[0, 0].set_ylabel('PSD')
    axs[1, 0].set_ylabel('mag. of FFT of PSD')

# Angle analysis

In [ ]:
from wall_analysis import filter_by_dicts
try:
    filter_dict = dict(
        distance=40,
        degree=0,
        source='sweep'
    )
    df = filter_by_dicts(df_total, [filter_dict])
    row = df.iloc[0]
    
    spec = np.sum(np.abs(row.stft), axis=1)
    
    plt.figure()
    times = np.arange(spec.shape[0])
    plt.pcolormesh(times, frequencies, np.log10(spec.T))

    psd = get_psd(row.stft, frequencies, ax=plt.gca(), fname='real')

    plt.figure()
    for i_mic in range(psd.shape[0]):
        plt.semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD')
    plt.title(filter_dict)
except ValueError:
    raise
except Exception as e:
    print(f'did not find {filter_dict} in')
    print(df_total.distance.unique())
    print(df_total.degree.unique())
    print(df_total.source.unique())
    print(e)

In [ ]:
degrees = [d for d in df_total.degree.unique() if d != 360][:2]
distances = df_total.distance.unique()[:3]
source = 'sweep'
print(distances, degrees)

fig, axs = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True, squeeze=False)
fig.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])

fig_psd, axs_psd = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True, squeeze=False)
fig_psd.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])
for i, distance in enumerate(distances):
    for j, degree in enumerate(degrees):
        df_this = df_total.loc[(df_total.distance == distance)
                               & (df_total.degree == degree)
                               & (df_total.source == source)]
        
        if not len(df_this) == 1:
            print('skipping', distance, degree, source)
            continue
        row = df_this.iloc[0]
        spec = row.spec #np.mean(row.spec, axis=1)
        psd = row.psd
        if psd is None:
            psd = get_psd(row.stft)
        
        axs[i, j].pcolorfast(range(spec.shape[0]), frequencies, np.log10(spec.T))
        for i_mic in range(psd.shape[0]):
            axs_psd[i, j].semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
        axs[0, j].set_title(f'{degree} deg')
        axs_psd[0, j].set_title(f'{degree} deg')
    axs[i, 0].set_ylabel(f'{distance} cm')
    axs_psd[i, 0].set_ylabel(f'{distance} cm')
    
[axs[-1, j].set_xlabel(f'time idx') for j in range(len(degrees))]
[axs_psd[-1, j].set_xlabel(f'frequency [Hz]') for j in range(len(degrees))]

In [ ]:
from constants import SPEED_OF_SOUND

distances = df_total.distance.unique()[:3]
distance_ref = sorted(df_total.distance.unique())[-1]
source = 'sweep'
n_mics = df_total.iloc[0].stft.shape[1]
print('n_mics:', n_mics)
    
for degree in [0, 27, 54, 81][:2]:
    
    df_ref = df_total.loc[(df_total.distance == distance_ref)
                           & (df_total.degree == degree)
                           & (df_total.source == source)]
    if len(df_ref) == 0:
        continue

    fig, axs = plt.subplots(1, n_mics, sharex=True, sharey=True)
    fig.set_size_inches(15, 5)
    
    for mic in range(n_mics):
        psd_ref = df_ref.iloc[0].psd[mic] 
        for i, distance in enumerate(distances):
            df_this = df_total.loc[(df_total.distance == distance)
                                   & (df_total.degree == degree)
                                   & (df_total.source == source)]
            axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic], label=distance, color=f"C{i}")
            #axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic] / psd_ref, label=distance, color=f"C{i}")
            #axs[mic].plot(frequencies, df_this.iloc[0].psd[mic] - psd_ref, label=distance, color=f"C{i}")

        #axs[mic].set_xlim(min(frequencies), max(frequencies))
        axs[mic].set_xlim(2000,  max(frequencies))
        axs[mic].set_title(f"mic{mic}")
    
    fig.suptitle(degree)
    axs[mic].legend()

In [ ]:
#chosen_frequencies = frequencies[[10, 20, 30]]
chosen_frequencies = frequencies[[30]]
print(chosen_frequencies)

distances = df_total.distance.unique()

mic = 0

for degree, df in df_total.groupby('degree'):
    source = 'sweep'

    fig, ax = plt.subplots()
    fig.set_size_inches(15, 5)
    for i, distance in enumerate(distances):
        df_this = df.loc[(df.distance == distance)
                       & (df.source == source)]
        if len(df_this) == 0:
            continue
        row = df_this.iloc[0]
        for f, freq in enumerate(chosen_frequencies):
            chosen_idx = np.where(frequencies == freq)[0][0]
            ax.scatter(distance, row.psd[mic, chosen_idx], color=f"C{f}")
    ax.set_title(degree)
    #ax.set_yscale('log')
    ax.legend(chosen_frequencies)
    ax.set_xlabel('distance [cm]')

## Moving angle analysis

In [ ]:
#freq = 4125
#source = 'None'
source = 'mono4125'
degree = 360

if not degree in df_total.degree.unique():
    raise ValueError('cannot do moving angle analysis on this dataset')

distances = df_total.distance.unique()[:3]
chosen_idx = np.where(frequencies == freq)[0][0]

averages = []
for i, distance in enumerate(distances):
    fig, axs = plt.subplots(2)
    fig.set_size_inches(15, 5)
    
    ax = axs[0]
    df_this = df_total.loc[(df_total.distance == distance)
                           & (df_total.degree == degree)]
    df_this = df_this.loc[df_this.source == source]
                           
    stft = df_this.iloc[0].stft
    yaw =  df_this.iloc[0].yaw
    spec = df_this.iloc[0].spec

    for j in range(stft.shape[1]):
        ax.semilogy(range(stft.shape[0]), np.abs(stft[:, j, chosen_idx]), color=f"C{j}", label=f"mic{j}")

    yaw[np.isnan(yaw)] = 0
    axs[1].plot(range(stft.shape[0]), yaw)
    for deg in -np.arange(1, 5)*90:
        index = np.nanargmin(np.abs(yaw-deg))
        axs[1].axvline(x=index, color='C1')
        ax.axvline(x=index, color='C1')
        
    min_avg = 200
    max_avg = 300
    averages.append(np.sum(spec[min_avg:max_avg, chosen_idx], axis=0))
        
    ax.set_title(distance)
    ax.set_ylim(0.05, 10)
    ax.legend()
    [ax.grid() for ax in axs]
    
    
plt.figure()
plt.scatter(distances, averages)
plt.xlabel('distance [cm]')
plt.title(f'average PSD at {freq} Hz')